### Imports, files, functions and constraints

In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import math
import scipy
from scipy.signal import find_peaks
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots 
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd

In [2]:
file_path_london_aadf = r"/Users/ksamorek/Desktop/Python TIL/data assignment/dft_aadf_region_id_6.csv"                           # This is yearly count
file_path_london_aadf_direction = r"/Users/ksamorek/Desktop/Python TIL/data assignment/dft_aadfbydirection_region_id_6.csv"      # This is yearly count by direction (2 ways included)
file_path_london_countpoints = r"/Users/ksamorek/Desktop/Python TIL/data assignment/dft_countpoints_region_id_6.csv"             # This doesnt have any data, it is just the countpoint coordinates
file_path_london_road_type = r"/Users/ksamorek/Desktop/Python TIL/data assignment/dft_traffic_road_type_region_id_6.csv"         # This is the Road Class 
file_path_london_vehicletype = r"/Users/ksamorek/Desktop/Python TIL/data assignment/dft_traffic_vehicle_type_region_id_6.csv"    # This is only for the whole london count

df_count = pd.read_csv(file_path_london_aadf)
df_count_per_side = pd.read_csv(file_path_london_aadf_direction)
df_countpoints = pd.read_csv(file_path_london_countpoints)
df_road_class = pd.read_csv(file_path_london_road_type)
df_london_count = pd.read_csv(file_path_london_vehicletype)

pd.set_option('display.max_columns', None)

In [3]:
intervals = [(2000, 2002), (2003, 2004), (2005, 2010), (2011, 2013), (2014, 2019), (2020, 2022)]

def map_to_interval(year):
    for interval in intervals:
        if interval[0] <= year <= interval[1]:
            return f"{interval[0]}-{interval[1]}"    

In [4]:
areas_inside_congestion_zone = ["Westminster", "Lambeth", "Southwark", "City of London",
                                 "Tower Hamlets", "Islington", "Camden"]                   
roads_to_remove = ["A501", "A40", "A5", "A4202", "A4", "A302", "A3204", "A3", "A201", 
                   "Tower Bridge Rd", "A100", "A1210", "Great Eastern St"]           

In [5]:
df_congestion = df_count[(df_count["local_authority_name"].isin(areas_inside_congestion_zone)) 
                         & (~df_count["road_name"].isin(roads_to_remove))]                    

In [6]:
df_yearly_car_type = df_congestion.groupby('year').agg({
    'pedal_cycles': 'sum',
    'two_wheeled_motor_vehicles': 'sum',
    'cars_and_taxis': 'sum',
    'buses_and_coaches': 'sum',
    'lgvs': 'sum',
    'all_hgvs': 'sum'
}).reset_index()

###  Count of all Motor Vehicles inside the Congestion Zone between change of prices 

In [10]:
df_yearly_congestion = df_congestion.groupby(["year"])["all_motor_vehicles"].sum().reset_index()



df_interval_congestion = df_yearly_congestion.groupby('year')['all_motor_vehicles'].mean().reset_index()

fig = px.bar(df_interval_congestion, x='year', y='all_motor_vehicles',
             labels={"all_motor_vehicles": "Average Total Motor Vehicles", "year_interval": "Year Interval"},
             title='Count of all motor vehicles inside the Congestion Zone over the years')
fig.update_xaxes(tickmode='array', tickvals=df_yearly_congestion['year'], ticktext=df_yearly_congestion['year'])
fig.update_layout(title=dict(x=0.5,y=0.84, xanchor='center'))
fig.update_xaxes(title="Year")
fig.update_yaxes(title="Total number of vehicles")
fig.show()



### Yearly distribution - Vechicle Type

In [11]:
df_yearly_car_type['year_interval'] = df_yearly_car_type['year'].apply(map_to_interval)

df_yearly_car_type

import plotly.express as px

fig = px.line(df_yearly_car_type, x='year', y=df_yearly_car_type.columns[1:-1],
              title='Vehicle Types Distribution Over Time',
              labels={'year': 'Year', 'value': 'Number of Vehicles'},
              line_dash_sequence=['solid', 'dash', 'solid', 'dash', 'solid', 'dash'],
              color_discrete_sequence=['blue', 'orange', 'green', 'red', 'purple', 'brown'],
              height=1000, width=1000)

fig.update_xaxes(tickmode='array', tickvals=df_yearly_car_type['year'], ticktext=df_yearly_car_type['year'])

fig.update_traces(name= 'Pedal cycles', selector=dict(name='pedal_cycles'))
fig.update_traces(name= 'Two-wheeled motor vechicles', selector=dict(name='two_wheeled_motor_vehicles'))
fig.update_traces(name= 'Cars (including taxis)', selector=dict(name='cars_and_taxis'))
fig.update_traces(name= 'Buses', selector=dict(name='buses_and_coaches'))
fig.update_traces(name= 'Large Goods Vehicle (LGV)', selector=dict(name='lgvs'))
fig.update_traces(name= 'Heavy Goods Vehicle (HGV)', selector=dict(name='all_hgvs'))

fig.update_layout(legend=dict(title_text='Vechicle type:', title_font=dict(size=12), orientation='v', y=0.99, x=0.855, xanchor='center', yanchor='top'), title=dict(x=0.5, y=0.917, xanchor='center'))
fig.show()